In [1]:
%pip install gqlalchemy python-dotenv yfinance

  Using cached tzdata-2025.3-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached pyjwt-2.11.0-py3-none-any.whl.metadata (4.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 25.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 29.8 MB/s eta 0:00:00
Using cached tzdata-2025.3-py2.py3-none-any.whl (348 kB)
Using cached pyjwt-2.11.0-py3-none-any.whl (28 kB)
  Attempting uninstall: psutil
    Found existing installation: psutil 7.2.2
    Uninstalling psutil-7.2.2:
      Successfully uninstalled psutil-7.2.2
  Attempting uninstall: neo4j
    Found existing installation: neo4j 6.1.0
    Uninstalling neo4j-6.1.0:
      Successfully uninstalled neo4j-6.1.0
  Attempting uninstall: pandas
    Found existing installation: pandas 3.0.1
    Uninstalling pandas-3.0.1:
      Successfully uninstalled pandas-3.0.1

[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart t

In [ ]:
import os
import yfinance as yf
from gqlalchemy import Memgraph
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# ───────────────────────────────────────────────
# 1. Connection Config
# ───────────────────────────────────────────────
HOST = "localhost"
PORT = 7687
USERNAME = os.getenv("MEMGRAPH_USERNAME", "memgraph")  # default is usually no auth
PASSWORD = os.getenv("MEMGRAPH_PASSWORD")              # can be None if no auth is set

# Connect to Memgraph
memgraph = Memgraph(
    host=HOST,
    port=PORT,
    #username=None,          # or "" if you prefer — None is fine
    #password=None,          # explicitly None or omit if no auth
    encrypted=False         # ← this is the key line for local setups
)

def ingest_financial_data(tickers):
    for symbol in tickers:
        # Fetch real data from Yahoo Finance
        stock = yf.Ticker(symbol)
        info = stock.info
        
        # Extract basic entities
        name = info.get('longName', symbol)
        sector = info.get('sector', 'Unknown')
        summary = info.get('longBusinessSummary', 'No summary available.')

        # Create Nodes and Relationships using Cypher
        query = """
        MERGE (c:Company {ticker: $ticker})
        SET c.name = $name, c.summary = $summary
        MERGE (s:Sector {name: $sector})
        MERGE (c)-[:IN_SECTOR]->(s)
        """
        
        memgraph.execute(
            query,
            {
                "ticker": symbol,
                "name": name,
                "sector": sector,
                "summary": summary
            }
        )
        
        print(f"Ingested: {name} in {sector} sector.")


def graph_rag_traversal(target_sector):
    """
    Example of 'Graph Retrieval Fusion':
    Find all companies in a sector and retrieve their summaries for RAG context.
    """
    query = """
    MATCH (s:Sector {name: $sector})<-[:IN_SECTOR]-(c:Company)
    RETURN c.name AS company, c.summary AS context
    """
    
    # Execute and get results
    results = memgraph.execute_and_fetch(query, {"sector": target_sector})
    
    # results is an iterator of dicts → convert to list of formatted strings
    return [
        f"Data for {r['company']}: {r['context'][:200]}..."
        for r in results
    ]



Connection failed: Function 'mg.version' doesn't exist.
Ingested: Apple Inc. in Technology sector.
Ingested: Microsoft Corporation in Technology sector.
Ingested: Alphabet Inc. in Communication Services sector.
Ingested: Tesla, Inc. in Consumer Cyclical sector.

Retrieved context:
Data for Apple Inc.: Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal ...
Data for Microsoft Corporation: Microsoft Corporation develops and supports software, services, devices, and solutions worldwide. The Productivity and Business Processes segment offers Microsoft 365 commercial, enterprise mobility +...


In [6]:

# ───────────────────────────────────────────────
# Example usage
# ───────────────────────────────────────────────

# Optional: test connection
try:
    version = list(memgraph.execute_and_fetch("RETURN mg.version() AS ver"))[0]["ver"]
    print(f"Connected to Memgraph version: {version}")
except Exception as e:
    print(f"Connection failed: {e}")
    exit(1)

tickers = ["AAPL", "MSFT", "GOOGL", "TSLA"]
ingest_financial_data(tickers)

# Test retrieval
sector = "Technology"  # ← make sure case matches exactly what was ingested
contexts = graph_rag_traversal(sector)

print("\nRetrieved context:")
if not contexts:
    print("No companies found in sector:", sector)
else:
    for line in contexts:
        print(line)

Connection failed: Function 'mg.version' doesn't exist.
Ingested: Apple Inc. in Technology sector.
Ingested: Microsoft Corporation in Technology sector.
Ingested: Alphabet Inc. in Communication Services sector.
Ingested: Tesla, Inc. in Consumer Cyclical sector.

Retrieved context:
Data for Apple Inc.: Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal ...
Data for Microsoft Corporation: Microsoft Corporation develops and supports software, services, devices, and solutions worldwide. The Productivity and Business Processes segment offers Microsoft 365 commercial, enterprise mobility +...
